# JsonToDictation

> Convert AWS Transcribe JSON format to Dictation Format

In [94]:
#| default_exp core

In [95]:
#| hide
from nbdev.showdoc import *
import json

In [96]:
#| export
class JsonToDictation:
    "A class that will convert the AWS Transcribe to Dictation"

    def __init__(self,
                 speakers,
                 data # JSON data that has been generated by AWS Transcribe
                ):
        self.transcribe_data = data
        self.titles = []
        self.num_spekaers = 0
        self.speakers = speakers
        self.speaker_map = []
        self.text = ''
        self.load()


    def load(self):
        self.titles = self.transcribe_data["results"]["transcripts"]
        self.num_speakers = self.transcribe_data["results"]["speaker_labels"]["speakers"] 
        self.getSpeakerMap()
        
    def getSpeakerMap(self):
        for segment in transcribe_json["results"]["speaker_labels"]["segments"]:
            speaker = segment["speaker_label"]            
            self.speaker_map.append(speaker)        

    def convert(self):
        str = ''
        current_speaker = ''
        for item in self.transcribe_data["results"]["items"]:
            alternatives = item["alternatives"]
            speaker_label = item["speaker_label"]
            speaker_index = self.speaker_map.index(speaker_label)
            speaker = self.speakers[speaker_index]
            word = alternatives[0]["content"]
            type = item["type"]

            if current_speaker != speaker:
                if current_speaker != '':
                    str += "\n\n"
                    
                current_speaker = speaker                
                str += f"{speaker}:\n"

            if type == "punctuation":
                str = str[:-1]
            
            str += f"{word} "
            
        self.text = str        
        return (str)        
            
    def summary(self):
        print(f"{self.titles=}")
        print(f"{self.num_speakers=}")
        print(f"{self.speakers=}")
        print(f"{self.speaker_map=}")


In [97]:
#| hide
import nbdev; nbdev.nbdev_export()

In [98]:
with open('data/transcribe_example.json') as transcribe_file:
  transcribe_json = json.load(transcribe_file)

speakers = ("BOB","JANE")
dictation = JsonToDictation(speakers,transcribe_json)
dictation.summary()

assert(dictation.num_speakers==2)

self.titles=[{'transcript': "I've been on hold for an hour. Sorry about that."}]
self.num_speakers=2
self.speakers=('BOB', 'JANE')
self.speaker_map=['spk_0', 'spk_1']


In [99]:
text = dictation.convert()
print (text)
assert(text=="BOB:\nI've been on hold for an hour. \n\nJANE:\nSorry about that. ")

BOB:
I've been on hold for an hour. 

JANE:
Sorry about that. 


In [74]:
print(json.dumps(transcribe_json,indent=2))

{
  "jobName": "my-first-transcription-job",
  "accountId": "111122223333",
  "results": {
    "transcripts": [
      {
        "transcript": "I've been on hold for an hour. Sorry about that."
      }
    ],
    "speaker_labels": {
      "channel_label": "ch_0",
      "speakers": 2,
      "segments": [
        {
          "start_time": "4.87",
          "speaker_label": "spk_0",
          "end_time": "6.88",
          "items": [
            {
              "start_time": "4.87",
              "speaker_label": "spk_0",
              "end_time": "5.02"
            },
            {
              "start_time": "5.02",
              "speaker_label": "spk_0",
              "end_time": "5.17"
            },
            {
              "start_time": "5.17",
              "speaker_label": "spk_0",
              "end_time": "5.29"
            },
            {
              "start_time": "5.29",
              "speaker_label": "spk_0",
              "end_time": "5.64"
            },
            {
 